In [268]:
import pandas as pd
from google.cloud import firestore
from google.cloud import bigquery
from datetime import datetime, timezone

teams = {'HomeTeam':'Boston Celtics', 'AwayTeam':'Detroit Pistons', 'Model':'automl_regression'}

df = pd.DataFrame(teams, index=[0])

model = df['Model'][0]

In [269]:
client = bigquery.Client()

inputs = client.query('''
    SELECT
      input
    FROM
      ML.FEATURE_INFO(MODEL `nba.%s`)
''' % (model)).to_dataframe()

In [270]:
inputs

,input
0,incoming_is_win_streak
1,incoming_wma_10_bench_plus_minus
2,incoming_wma_10_efg_pct
3,incoming_wma_10_ft_rate
4,incoming_wma_10_off_rtg
5,incoming_wma_10_opponent_bench_plus_minus
6,incoming_wma_10_opponent_efg_pct
7,incoming_wma_10_opponent_ft_rate
8,incoming_wma_10_opponent_off_rtg
9,incoming_wma_10_opponent_tov_pct


In [271]:
db = firestore.Client()

home_team_data = db.collection('team_model_data').document(df['HomeTeam'][0]).get().to_dict()
away_team_data = db.collection('team_model_data').document(df['AwayTeam'][0]).get().to_dict()

In [272]:
home_team_data

{'wma_10_opponent_bench_plus_minus': 0.8545454545454545,
 'wma_10_starter_minutes_played_proportion': 0.6255297371675052,
 'streak_counter_is_win': 4,
 'wma_10_off_rtg': 121.73818181818181,
 'wma_10_opponnent_starter_minutes_played_proportion': 0.6348266359991303,
 'wma_10_opponent_ft_rate': 0.2048727272727273,
 'wma_10_ft_rate': 0.24576363636363635,
 'wma_10_tov_pct': 11.516363636363637,
 'wma_10_pace': 95.96363636363637,
 'wma_10_opponent_off_rtg': 120.3509090909091,
 'wma_10_efg_pct': 0.5697454545454546,
 'wma_10_opponent_efg_pct': 0.5599454545454545,
 'wma_10_opponent_tov_pct': 10.994545454545456,
 'season': 2021,
 'game_date': DatetimeWithNanoseconds(2021, 3, 4, 0, 0, tzinfo=<UTC>),
 'wma_10_bench_plus_minus': -2.9272727272727272}

In [273]:
away_team_data

{'wma_10_opponent_bench_plus_minus': -17.545454545454547,
 'season': 2021,
 'wma_10_off_rtg': 112.00545454545455,
 'wma_10_starter_minutes_played_proportion': 0.5988811190767316,
 'streak_counter_is_win': -1,
 'wma_10_pace': 95.77454545454546,
 'wma_10_opponent_off_rtg': 114.65454545454546,
 'wma_10_opponent_tov_pct': 11.156363636363636,
 'wma_10_bench_plus_minus': 6.036363636363636,
 'game_date': DatetimeWithNanoseconds(2021, 3, 4, 0, 0, tzinfo=<UTC>),
 'wma_10_ft_rate': 0.19163636363636363,
 'wma_10_efg_pct': 0.5419090909090909,
 'wma_10_opponent_ft_rate': 0.22289090909090908,
 'wma_10_opponent_efg_pct': 0.5352545454545454,
 'wma_10_tov_pct': 12.341818181818182,
 'wma_10_opponnent_starter_minutes_played_proportion': 0.6610975265223425}

In [274]:
for key in home_team_data.keys():

SyntaxError: unexpected EOF while parsing (<ipython-input-274-c03b4156a048>, line 1)

In [275]:
datetime.now(timezone.utc)

datetime.datetime(2021, 3, 11, 5, 41, 47, 861755, tzinfo=datetime.timezone.utc)

In [276]:
if 'rest_days_difference' in inputs.values:
    home_rest = (datetime.now(timezone.utc) - home_team_data['game_date']).days
    away_rest = (datetime.now(timezone.utc) - away_team_data['game_date']).days
    rest_days_difference = home_rest - away_rest

In [277]:
rest_days_difference

0

In [278]:
for column in inputs.input:
    print (column)

incoming_is_win_streak
incoming_wma_10_bench_plus_minus
incoming_wma_10_efg_pct
incoming_wma_10_ft_rate
incoming_wma_10_off_rtg
incoming_wma_10_opponent_bench_plus_minus
incoming_wma_10_opponent_efg_pct
incoming_wma_10_opponent_ft_rate
incoming_wma_10_opponent_off_rtg
incoming_wma_10_opponent_tov_pct
incoming_wma_10_opponnent_starter_minutes_played_proportion
incoming_wma_10_pace
incoming_wma_10_starter_minutes_played_proportion
incoming_wma_10_tov_pct
is_home_team


In [279]:
inputs[inputs['input'] != 'is_home_team']

,input
0,incoming_is_win_streak
1,incoming_wma_10_bench_plus_minus
2,incoming_wma_10_efg_pct
3,incoming_wma_10_ft_rate
4,incoming_wma_10_off_rtg
5,incoming_wma_10_opponent_bench_plus_minus
6,incoming_wma_10_opponent_efg_pct
7,incoming_wma_10_opponent_ft_rate
8,incoming_wma_10_opponent_off_rtg
9,incoming_wma_10_opponent_tov_pct


In [280]:
query = f'SELECT predicted_spread FROM ML.PREDICT(MODEL `nba.{model}`, (SELECT '

for column in inputs.input:
    key = column[9:]
    if column == 'is_home_team':
        query = query + '1 as is_home_team,'
    elif column == 'rest_days_difference':
        home_rest = (datetime.now(timezone.utc) - home_team_data['game_date']).days
        away_rest = (datetime.now(timezone.utc) - away_team_data['game_date']).days
        rest_days_difference = home_rest - away_rest
        query = query + f'{rest_days_difference} as {column},'
    elif column == 'incoming_is_win_streak':
        query = query + f'{home_team_data["streak_counter_is_win"]} as {column},'
    elif (column == 'opponent_incoming_is_win_streak') | (column == 'incoming_is_win_streak_opponent'):
        query = query + f'{away_team_data["streak_counter_is_win"]} as {column},'
    elif column[:12] == 'incoming_wma':
        if column.split('_')[3] == 'opponent':
            query = query + f'{away_team_data[key]} as {column},'
        else:
            query = query + f'{away_team_data[key]} as {column},'
    else:
        print(f'Error: Model input column {column} not in team data in firestore or not in logic in App Engine. Please try a different model')

In [281]:
bq_query = query[:-1] + '))'

game = client.query('''
%s
''' % (bq_query)).to_dataframe()

pointspread = round(game['predicted_spread'][0],1)

if pointspread > 0:
    winner = df['HomeTeam'][0]
    loser = df['AwayTeam'][0]
else:
    winner = df['AwayTeam'][0]
    loser = df['HomeTeam'][0]

In [282]:
f'I predict the {winner} will beat the {loser} by {abs(pointspread)} points using the {model} model!'

'I predict the Boston Celtics will beat the Detroit Pistons by 1.3 points using the automl_regression model!'

In [212]:
os.environ['GCP_PROJECT'] = 'nba-predictions-dev'

In [209]:
from google.cloud import bigquery

client = bigquery.Client(project='nba-predictions-dev')

dataset_id = 'nba'

models = client.list_models(dataset_id) 

model_names = [model.model_id for model in models]

# print("Models contained in '{}':".format(dataset_id))
# for model in models:
#     full_model_id = "{}.{}.{}".format(
#         model.project, model.dataset_id, model.model_id
#     )
#     friendly_name = model.friendly_name
#     print("{}: friendly_name='{}'".format(full_model_id, friendly_name))

In [210]:
model_names = [model.model_id for model in models]

In [211]:
model_names

['BOOSTED_TREE_CLASSIFIER',
 'BOOSTED_TREE_CLASSIFIER_ALL_VARIABLES',
 'BOOSTED_TREE_CLASSIFIER_ALL_VARIABLES_DIFFERENCES',
 'automl_classification',
 'automl_regression',
 'baseline_linear_model']

In [207]:
model_names = []
for model in models:
    model_names.append(model.model_id)

In [208]:
model_names

['nba-predictions-dev.nba.BOOSTED_TREE_CLASSIFIER',
 'nba-predictions-dev.nba.BOOSTED_TREE_CLASSIFIER_ALL_VARIABLES',
 'nba-predictions-dev.nba.BOOSTED_TREE_CLASSIFIER_ALL_VARIABLES_DIFFERENCES',
 'nba-predictions-dev.nba.automl_classification',
 'nba-predictions-dev.nba.automl_regression',
 'nba-predictions-dev.nba.baseline_linear_model']

In [204]:
model_names = [f'{model.project}.{model.dataset_id}.{model.model_id}' for model in models]

ValueError: ('Iterator has already started', <google.api_core.page_iterator.HTTPIterator object at 0x000001FB97255E80>)

In [199]:
from google.cloud import bigquery

# Construct a BigQuery client object.
#client = bigquery.Client()

# TODO(developer): Set model_id to the ID of the model to fetch.
model_id = 'nba-predictions-dev.nba.BOOSTED_TREE_CLASSIFIER'

model = client.get_model(model_id)  # Make an API request.

full_model_id = "{}.{}.{}".format(model.project, model.dataset_id, model.model_id)
friendly_name = model.friendly_name
print(
    "Got model '{}' with friendly_name '{}'.".format(full_model_id, friendly_name)
)

Got model 'nba-predictions-dev.nba.BOOSTED_TREE_CLASSIFIER' with friendly_name 'None'.


In [200]:
model

Model(reference=ModelReference(project_id='nba-predictions-dev', dataset_id='nba', model_id='BOOSTED_TREE_CLASSIFIER'))

In [ ]:
## Run this function locally after initial data load since it will take more memory than google cloud functions allows

import pandas as pd
import numpy as np
from google.cloud import bigquery
from google.cloud import firestore
import os

## TO DO: Replace your Project ID and change the table names if you chose a different dataset name than 'nba' ##

os.environ['GCP_PROJECT'] = 'nba-predictions-test'
my_project_id = os.environ.get('GCP_PROJECT')
client = bigquery.Client(project=my_project_id)
raw_game_data_table = f'{my_project_id}.nba.raw_basketballreference_game'
raw_player_data_table = f'{my_project_id}.nba.raw_basketballreference_playerbox'
games_to_load_to_model_view = f'{my_project_id}.nba.games_to_load_to_model'
model_table_name = f'{my_project_id}.nba.model_game'

# Enter columns you want to generate linearly weighted moving average calculations for and number of periods to use
wma_columns = [
    'pace','efg_pct', 'tov_pct', 'ft_rate', 'off_rtg',
    'opponent_efg_pct', 'opponent_tov_pct', 'opponent_ft_rate',
    'opponent_off_rtg', 'starter_minutes_played_proportion',
    'bench_plus_minus', 'opponnent_starter_minutes_played_proportion',
    'opponent_bench_plus_minus']
W = 20

In [178]:
def convert_to_seconds(x):
    sp = int(x.split(':')[0]) * 60 + int(x.split(':')[1])
    return sp

def switch_key(key):
    new_key = key[:-1] + ('h' if key[-1] == 'a' else 'a')
    return new_key

def generate_streak_info(data,column):
    """
    Parameters
    ----------
    data:
      Dataframe with a specific column to generate streak data

    column:
      Stirng with specific column name to generate streak info

    Returns
    -------

    data_with_streak_counter:
        The original dataframe with a new column
        `streak_counter_[column]` containing integers with 
        counts for each streak.
    """
    
    data['start_of_streak'] = data[column].ne(data[column].shift())
    data['streak_id'] = data.start_of_streak.cumsum()
    data[f'streak_counter_{column}'] = data.groupby('streak_id').cumcount() + 1
    data_with_streak_counter = data.drop(columns = ['start_of_streak','streak_id'] )
    return data_with_streak_counter

def create_linear_weighted_moving_average(data,column,weight):
    """
    Parameters
    ----------
    data:
      Dataframe with a specific column to generate weighted moving average.

    column:
      Stirng with specific column name to generate weighted moving average info.
      Column must be ready to be converted to float data type.

    Returns
    -------

    data_with_moving_average:
        The original dataframe with a new column
        `wma_[W]_[column]` containing float values with weighted moving average
        values for the provided value with a weight of W.
    """  
    data_with_moving_average = data.copy()
    data_with_moving_average[column] = data_with_moving_average[column].astype(float)
    weights = np.arange(1,weight+1)
    data_with_moving_average[f'wma_{weight}_{column}'] = data_with_moving_average[column].rolling(weight).apply(lambda col: np.dot(col, weights)/weights.sum(), raw=True)
    return data_with_moving_average

## Load tables to dataframe
game_bq = client.query('''
SELECT game_date, visitor_team_name, visitor_pts, home_team_name, home_pts, games.game_key, 
    a_ff_pace, a_ff_efg_pct, a_ff_tov_pct, a_ff_orb_pct, a_ff_ft_rate, a_ff_off_rtg, 
    h_ff_pace, h_ff_efg_pct, h_ff_tov_pct,h_ff_orb_pct, h_ff_ft_rate, h_ff_off_rtg
FROM `%s` as games
''' % (raw_game_data_table)).to_dataframe()

if game_bq.empty:
    print('Function ended early. No new data to load.')

player_bq = client.query('''
SELECT players.game_key, game_date, h_or_a, mp, plus_minus, starter_flag
FROM `%s` as players
WHERE mp is not NULL
''' % (raw_player_data_table)).to_dataframe()

## Create copies to avoid calling bigquery multiple times when testing - comment out delete while testing
game = game_bq.copy()
player = player_bq.copy()

#del game_bq
#del player_bq

## Create game variables needed for model
game['home_spread'] = game['home_pts'].astype(int) - game['visitor_pts'].astype(int)
game['season'] = ''
for i in range(len(game)):
    if ((game['game_date'][i].year != 2020 and game['game_date'][i].month < 7) or (game['game_date'][i].year == 2020 and game['game_date'][i].month < 11)):
        game.loc[i,'season'] = game['game_date'][i].year
    else:
        game.loc[i,'season'] = game['game_date'][i].year + 1


## Create game by team variables - stack home and away to team vs. opponent
games_by_team_home = pd.DataFrame()
games_by_team_home['season'] = game['season']
games_by_team_home['game_key'] = game['game_key']
games_by_team_home['game_key_team'] = game['game_key'] + 'h'
games_by_team_home['game_key_opponent'] = game['game_key'] + 'a'
games_by_team_home['game_date'] = pd.to_datetime(game['game_date'])
games_by_team_home['team'] = game['home_team_name']
games_by_team_home['opponent'] = game['visitor_team_name']
games_by_team_home['is_home_team'] = 1
games_by_team_home['spread'] = game['home_spread']
games_by_team_home['pace'] = game['h_ff_pace']
games_by_team_home['efg_pct'] = game['h_ff_efg_pct']
games_by_team_home['tov_pct'] = game['h_ff_tov_pct']
games_by_team_home['ft_rate'] = game['h_ff_ft_rate']
games_by_team_home['off_rtg'] = game['h_ff_off_rtg']
games_by_team_home['opponent_efg_pct'] = game['a_ff_efg_pct']
games_by_team_home['opponent_tov_pct'] = game['a_ff_tov_pct']
games_by_team_home['opponent_ft_rate'] = game['a_ff_ft_rate']
games_by_team_home['opponent_off_rtg'] = game['a_ff_off_rtg']
games_by_team_home['is_win'] = [1 if x > 0 else 0 for x in games_by_team_home['spread'].astype(int)]


games_by_team_visitor = pd.DataFrame()
games_by_team_visitor ['season'] = game['season']
games_by_team_visitor['game_key'] = game['game_key']
games_by_team_visitor ['game_key_team'] = game['game_key'] + 'a'
games_by_team_visitor ['game_key_opponent'] = game['game_key'] + 'h'
games_by_team_visitor ['game_date'] = pd.to_datetime(game['game_date'])
games_by_team_visitor ['team'] = game['visitor_team_name']
games_by_team_visitor ['opponent'] = game['home_team_name']
games_by_team_visitor ['is_home_team'] = 0
games_by_team_visitor ['spread'] = game['home_spread']*-1
games_by_team_visitor ['pace'] = game['a_ff_pace']
games_by_team_visitor ['efg_pct'] = game['a_ff_efg_pct']
games_by_team_visitor ['tov_pct'] = game['a_ff_tov_pct']
games_by_team_visitor ['ft_rate'] = game['a_ff_ft_rate']
games_by_team_visitor ['off_rtg'] = game['a_ff_off_rtg']
games_by_team_visitor['opponent_efg_pct'] = game['h_ff_efg_pct']
games_by_team_visitor['opponent_tov_pct'] = game['h_ff_tov_pct']
games_by_team_visitor['opponent_ft_rate'] = game['h_ff_ft_rate']
games_by_team_visitor['opponent_off_rtg'] = game['h_ff_off_rtg']
games_by_team_visitor['is_win'] = [1 if x > 0 else 0 for x in games_by_team_visitor['spread'].astype(int)]

games_by_team = pd.concat([games_by_team_home,games_by_team_visitor])

games_by_team.sort_values(by=['game_date'], ascending = True, inplace=True)

games_by_team['previous_game_date'] = games_by_team.groupby(['team'])['game_date'].shift(1)

games_by_team['incoming_rest_days'] = [(d - p).days for d,p in zip(games_by_team['game_date'],games_by_team['previous_game_date'])] 

# Replace NaN with -99 so can be converted to int. These rows will be dropped later.
games_by_team['incoming_rest_days'].fillna(-99, inplace=True)

games_by_team['incoming_rest_days'] = games_by_team['incoming_rest_days'].astype(int)

games_by_team.set_index('game_key_team', inplace=True)

# del games_by_team_visitor
# del games_by_team_home

## Create player variables needed for model
# Make game key unique per home/away team
player['game_key_team'] = player['game_key'] + player['h_or_a']

#Only include players that actually played
player = player.dropna(subset=['mp', 'plus_minus']).reset_index(drop=True)

player['plus_minus'] = player['plus_minus'].astype(int)
player['seconds_played'] = player['mp'].apply(convert_to_seconds)

## Create dataframe for aggregated player stats per game
game_player_stats = pd.DataFrame()
game_player_stats['game_key_team'] = player['game_key_team'].unique()

total_seconds = player.groupby(['game_key_team'])['seconds_played'].sum()
starter_seconds = player[player['starter_flag']==True].groupby(['game_key_team'])['seconds_played'].sum()
seconds = pd.merge(total_seconds, starter_seconds, left_index=True, right_index=True, how='inner')
seconds['starter_minutes_played_proportion'] = seconds['seconds_played_y']/seconds['seconds_played_x']

game_player_stats.set_index('game_key_team',inplace=True)
game_player_stats = pd.merge(game_player_stats,seconds['starter_minutes_played_proportion'],left_index=True,right_index=True,how='inner')

bench_pl_min = player[player['starter_flag']==False].groupby(['game_key_team'])['plus_minus'].sum()
game_player_stats = pd.merge(game_player_stats,bench_pl_min, left_index=True, right_index=True, how='inner')
game_player_stats = game_player_stats.rename(columns={'plus_minus':'bench_plus_minus'})

## Merge aggregated stats in to games by team dataframe
games_by_team = pd.merge(games_by_team,game_player_stats, left_index=True, right_index=True,how='inner')

## Create dataframe to capture opponent aggregated stats
game_player_stats_opponent = game_player_stats.copy()

# del game_player_stats

# Reset index so it can be modified to temporarily swith 'h' with 'a'
game_player_stats_opponent.reset_index(drop=False, inplace=True)
game_player_stats_opponent['game_key_team'] = game_player_stats_opponent['game_key_team'].apply(switch_key)

#Rename columns to opponent columns
game_player_stats_opponent = game_player_stats_opponent.rename(columns={'starter_minutes_played_proportion':'opponnent_starter_minutes_played_proportion','bench_plus_minus':'opponent_bench_plus_minus'})

#Reset index and merge
game_player_stats_opponent.set_index('game_key_team', inplace=True)
games_by_team = pd.merge(games_by_team,game_player_stats_opponent,left_index=True,right_index=True,how='inner')

# del game_player_stats_opponent

games_by_team_with_wma = pd.DataFrame()
#Create data frame with stats needed for model
for team in games_by_team['team'].unique():
    team_games = games_by_team.loc[games_by_team['team']==team].sort_values(by='game_date')
    team_games = generate_streak_info(team_games,'is_win')
    team_games['streak_counter_is_win'] = [x * -1 if y == 0 else x for x,y in zip(team_games['streak_counter_is_win'],team_games['is_win'])]
    team_games['incoming_is_win_streak'] = team_games['streak_counter_is_win'].shift(fill_value=0)
    for col in wma_columns:
        team_games = create_linear_weighted_moving_average(team_games,col,W)
        team_games[f'incoming_wma_{W}_{col}'] = team_games[f'wma_{W}_{col}'].shift()
    games_by_team_with_wma = pd.concat([games_by_team_with_wma, team_games])

games_by_team_with_wma = (games_by_team_with_wma.merge(games_by_team_with_wma.reset_index(drop=False)[[
                'game_key_team','incoming_rest_days','streak_counter_is_win','incoming_is_win_streak']],
                    left_on='game_key_opponent', right_on='game_key_team',
                    how='inner', suffixes=(None, '_opponent'))) 

#Drop first W rows for each team with no incoming weighted average
model_game_data = games_by_team_with_wma.dropna(subset=[f'incoming_wma_{W}_pace']).copy()

# del games_by_team_with_wma
# del games_by_team

#Convert data types to prepare for load to bigquery
model_game_data = model_game_data.astype({'season':int, 'is_win':int})

#Create data frame to create firestore collections with data to use in model call
most_recent_game = model_game_data.sort_values('game_date').drop_duplicates(['team'],keep='last')

most_recent_game = most_recent_game[['season', 'game_date', 'team','streak_counter_is_win']
                                                     + [f'wma_{W}_{x}' for x in wma_columns]]
most_recent_game.reset_index(drop=True, inplace=True)
most_recent_game.set_index('team', inplace=True)
docs = most_recent_game.to_dict(orient='index')

db = firestore.Client(project=my_project_id)
for team in most_recent_game.index.unique():
    doc_ref = db.collection('team_model_data').document(team.replace('/','\\')) #Teams that changed mid-season have a '/' which firestore interprets as new path
    doc_ref.set(docs[team])

# del most_recent_game

#Create new client and load table to Big Query
bqclient = bigquery.Client(project=my_project_id)
#Publish model data
job_config = bigquery.LoadJobConfig()
job_config.autodetect='True'
job_config.create_disposition = 'CREATE_IF_NEEDED'
job_config.write_disposition = 'WRITE_TRUNCATE'
# job_config.time_partitioning = bigquery.TimePartitioning(
#     type_=bigquery.TimePartitioningType.DAY,
#     field="game_date")
## Set schema for specific columns where more information is needed (e.g. not NULLABLE or specific date/time)
job_config.schema = [
    bigquery.SchemaField('game_key','STRING', 'REQUIRED'),
    bigquery.SchemaField('team','STRING', 'REQUIRED'),
    bigquery.SchemaField('opponent','STRING', 'REQUIRED'),
    bigquery.SchemaField('game_date','DATE'),
]
job_model = bqclient.load_table_from_dataframe(model_game_data, model_table_name, job_config=job_config)

model_result = job_model.result()
model_message = (
    f'Job ID: {model_result.job_id} '
    f'was started {model_result.started} '
    f'and ended {model_result.ended} '
    f'loading {model_result.output_rows} row(s) '
    f'to {model_result.destination}')

print(model_message)

In [ ]:
def remove_duplicates(table, distinct_column):
    ## Remove Duplicates - Always run at the end if you have had issues while loading ##
    client = bigquery.Client(project=os.environ.get('GCP_PROJECT'))

    ## Get row count and distinct game count
    game_count = client.query('''
    select count(1) as row_count,
        count(distinct %s) as game_count 
        from `%s.%s`
    ''' % (distinct_column,os.environ.get('GCP_PROJECT'),table)).to_dataframe()

    if game_count['row_count'][0] == game_count['game_count'][0]:
        return f'No duplicates in {table}!'
    else:
        deduplicate = client.query('''
        CREATE OR REPLACE TABLE `%s.%s`
        AS
        SELECT * EXCEPT(row_num) FROM (
        SELECT
        *, ROW_NUMBER() OVER (PARTITION BY %s ORDER BY load_datetime desc) as row_num
        FROM `%s.%s`
        ) WHERE row_num = 1
        ''' % (os.environ.get('GCP_PROJECT'),table,distinct_column,os.environ.get('GCP_PROJECT'),table))
        return 'Duplicates removed from {table}'

In [ ]:
game_remove = remove_duplicates('nba.raw_basketballreference_game','game_key')
player_remove = remove_duplicates('nba.raw_basketballreference_playerbox', 'player_stat_key')

print(game_remove)
print(player_remove)

In [ ]:
os.environ['GCP_PROJECT'] = 'nba-predictions-dev'

In [ ]:
import os
from google.cloud import bigquery

## Setup
my_project_id = os.environ.get('GCP_PROJECT')
client = bigquery.Client(project=my_project_id)
raw_game_data_table = f'{my_project_id}.nba.raw_basketballreference_game'
raw_player_data_table = f'{my_project_id}.nba.raw_basketballreference_playerbox'
games_to_load_to_model_view = f'{my_project_id}.nba.games_to_load_to_model'
model_table_name = f'{my_project_id}.nba.model_game'

# Enter columns to created linearly weighted moving average calculations and number of periods to use
wma_columns = ['pace',
    'efg_pct', 'tov_pct', 'ft_rate', 'off_rtg',
    'opponent_efg_pct', 'opponent_tov_pct', 'opponent_ft_rate',
    'opponent_off_rtg', 'starter_minutes_played_proportion',
    'bench_plus_minus', 'opponnent_starter_minutes_played_proportion',
    'opponent_bench_plus_minus']
W = 10

## Load tables to dataframe
game_bq = client.query('''
SELECT game_date, visitor_team_name, visitor_pts, home_team_name, home_pts, games.game_key, 
    a_ff_pace, a_ff_efg_pct, a_ff_tov_pct, a_ff_orb_pct, a_ff_ft_rate, a_ff_off_rtg, 
    h_ff_pace, h_ff_efg_pct, h_ff_tov_pct,h_ff_orb_pct, h_ff_ft_rate, h_ff_off_rtg
    ,NEEDS_TO_LOAD_TO_MODEL
FROM `%s` as games
INNER JOIN `%s` as load ON games.game_key = load.game_key 
''' % (raw_game_data_table,games_to_load_to_model_view)).to_dataframe()

In [ ]:
if game_bq.empty:
    print('empty')

In [ ]:
import pandas as pd
import numpy as np
from google.cloud import firestore
from google.cloud import bigquery

def predicted_pointspread(form_dict):
    try:
        teams = {'HomeTeam':'Atlanta Hawks', 'AwayTeam':'Milwaukee Bucks'}
        df = pd.DataFrame(teams, index=[0])


        db = firestore.Client()

        home_team_data = db.collection('team_model_data').document(df['HomeTeam'][0]).get().to_dict()
        away_team_data = db.collection('team_model_data').document(df['AwayTeam'][0]).get().to_dict()

        query = 'SELECT predicted_spread FROM ML.PREDICT(MODEL `nba.automl_regression`, (SELECT 1 as is_home_team,'
        for key in home_team_data.keys():
            if key == 'streak_counter_is_win':
                query = query + f'{home_team_data[key]} as incoming_is_win_streak,'
            elif key not in ['season', 'game_date']:
                query = query + f'{home_team_data[key]} as incoming_{key},'
        for key in away_team_data.keys():
            if key not in ['season', 'game_date', 'streak_counter_is_win']:
                query = query + f'{away_team_data[key]} as incoming_opponent_{key},'

        bq_query = query[:-1] + '))'

        client = bigquery.Client()

        game_bq = client.query('''
        %s
        ''' % (bq_query))

        game = game_bq.to_dataframe()

        pointspread = round(game['predicted_spread'][0],1)

        if pointspread > 0:
            winner = df['HomeTeam'][0]
            loser = df['AwayTeam'][0]
        else:
            winner = df['AwayTeam'][0]
            loser = df['HomeTeam'][0]
        return f'I predict the {winner} will beat the {loser} by {abs(pointspread)} points!'
    except Exception as e:
        raise ValueError('Sorry, there was a problem processing the data entered... Please try again with different teams') from e

In [ ]:
pointspread = np.random.randint(-1000,1000)
pointspread = 1 if pointspread == 0 else pointspread
if pointspread > 0:
    winner = df['HomeTeam'][0]
    loser = df['AwayTeam'][0]
else:
    winner = df['AwayTeam'][0]
    loser = df['HomeTeam'][0]
        return f'I predict the {winner} will beat the {loser} by {abs(pointspread)} points!'
    except Exception as e:
        raise ValueError('Sorry, there was a problem processing the data entered... Please go back and double check your entries, thanks!') from e

In [ ]:

from flask import Flask, render_template, request#, url_for, redirect
from google.cloud import storage
from google.cloud import firestore
import json
import os

In [ ]:
%env CLOUD_STORAGE_BUCKET nba-predictions-dev.appspot.com

In [ ]:
os.environ.get("CLOUD_STORAGE_BUCKET") 

In [ ]:
    client = storage.Client()
    bucket_name = os.environ.get("CLOUD_STORAGE_BUCKET") #'nba-predictions-dev.appspot.com'
    bucket = client.bucket(bucket_name)
    blob = bucket.blob('static/monday.json').download_as_string()
    data = json.loads(blob.decode("utf-8").replace("'",'"'))
    home_teams = list(data['home_team_name'].values())
    away_teams = list(data['visitor_team_name'].values())
    game_day = list(data['game_day'].values())
    game_date = list(data['game_date'].values())
    game_start_time = list(data['game_start_time'].values())
    games = []
    for i in range(len(home_teams)-1):
        games.append(f'{away_teams[i]} vs. {home_teams[i]} at {game_start_time[i]} on {game_day[i]}, {game_date[i]}')
    return render_template('UpcomingGames.html', games=games, home_teams = home_teams, away_teams=away_teams, game_day=game_day, game_date = game_date, game_start_time = game_start_time)


In [ ]:
import requests
from datetime import date, timedelta

In [ ]:
request = {"StartDate":"2015-02-01"}

In [ ]:
startDate = datetime.strptime(request['StartDate'], '%Y-%m-%d').date()

In [ ]:
endDate_max = (datetime.now() + timedelta(days=-1)).date()

In [ ]:
endDate = ''

In [ ]:
url = 'https://us-central1-nba-predictions-dev.cloudfunctions.net/nba_basketball_reference_scraper'

In [ ]:
#while endDate != endDate_max:
year, month= divmod(startDate.month+1, 12)
if month == 0: 
      month = 12
      year = year -1
endDate = date(startDate.year + year, month, 1) + timedelta(days=-1)
if endDate >= endDate_max:
    endDate = endDate_max
endDateformat = endDate.strftime('%Y-%m-%d')
startDateformat = startDate.strftime('%Y-%m-%d')
data = {"StartDate":startDateformat,"EndDate":endDateformat}
print(data)
# response = requests.post(url, data)
# print(response)
year, month= divmod(startDate.month+1, 12)
if month == 0: 
      month = 12
      year = year -1
startDate = date(startDate.year + year, month, 1)

In [ ]:
response = requests.post(url, data)
print(response)

In [ ]:
startDate

In [ ]:
form_dict = {'HomeTeam':'Atlanta Hawks', 'AwayTeam':'Boston Celtics'}

In [ ]:
import pandas as pd
import numpy as np

def predicted_pointspread(form_dict):
    try:
        df = pd.DataFrame(form_dict, index=[0])
        pointspread = np.random.randint(-40,40)
        pointspread = 1 if pointspread == 0 else pointspread
        if pointspread > 0:
            winner = df['HomeTeam'][0]
            loser = df['AwayTeam'][0]
        else:
            winner = df['AwayTeam'][0]
            loser = df['HomeTeam'][0]
        return f'I predict the {winner} will beat the {loser} by {abs(pointspread)} points'
    except:
        return 'Sorry, there was a problem processing the data entered... Please go back and double check your entries, thanks!'
        

In [ ]:
predicted_pointspread(form_dict)

In [ ]:
df = pd.DataFrame(form_dict, index=[0])

In [ ]:
df['HomeTeam'][0]

In [ ]:
import requests
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import pandas as pd
from google.cloud import bigquery
import pyarrow

request = {'StartDate': '2019-10-01'}

In [ ]:
def get_max_game_date():
    client = bigquery.Client(project=projcet)
    
    QUERY = (
    "SELECT date_add(max(game_date), INTERVAL 1 day) as max_game_date FROM `nba.raw_basketballreference_game`"
    )
    query_job = client.query(QUERY)  # API request
    rows = query_job.result()  # Waits for query to finish
    
    for result in rows:
        load_date = result.max_game_date
    
    return load_date

def get_game_players(soup, player_game_data, id_string, game_key, stat_type, h_or_a, team_abbrev, game_date):
    rows = soup.find('table', id=id_string).find('tbody').find_all('tr')
    cnt = 1

    #print(str(rows))
    for player in rows:
        game_players = {}
        game_players['game_key'] = game_key
        game_players['game_date'] = game_date
        game_players['h_or_a'] = h_or_a
        game_players['team_abbrev'] = team_abbrev
        game_players['stat_period'] = stat_type
        game_players['player'] = player.find('th',{"data-stat": "player"}).text
        #print(game_players['player'])
        
        player_node = player.find('th',{"data-stat": "player"})
        
        # Ignore Header Line
        if game_players['player'] != 'Reserves' and player_node.has_attr('data-append-csv'):
        
            a = player.find('th',{"data-stat": "player"}).find('a',href=True)
            if a is not None:
                game_players['player_link'] = a['href']
            else:
                game_players['player_link'] = None
            
            game_players['player_key'] = player_node['data-append-csv']
            game_players['reason'] = get_text(player.find('td',{"data-stat": "reason"}))
            game_players['mp'] = get_text(player.find('td',{"data-stat": "mp"}))
            game_players['fg'] = get_text(player.find('td',{"data-stat": "fg"}))
            game_players['fga'] = get_text(player.find('td',{"data-stat": "fga"}))
            game_players['fg_pct'] = get_text(player.find('td',{"data-stat": "fg_pct"}))
            game_players['fg3'] = get_text(player.find('td',{"data-stat": "fg3"}))
            game_players['fg3a'] = get_text(player.find('td',{"data-stat": "fg3a"}))
            game_players['fg3_pct'] = get_text(player.find('td',{"data-stat": "fg3_pct"}))
            game_players['ft'] = get_text(player.find('td',{"data-stat": "ft"}))
            game_players['fta'] = get_text(player.find('td',{"data-stat": "fta"}))
            game_players['ft_pct'] = get_text(player.find('td',{"data-stat": "ft_pct"}))
            game_players['orb'] = get_text(player.find('td',{"data-stat": "orb"}))
            game_players['drb'] = get_text(player.find('td',{"data-stat": "drb"}))
            game_players['trb'] = get_text(player.find('td',{"data-stat": "trb"}))
            game_players['ast'] = get_text(player.find('td',{"data-stat": "ast"}))
            game_players['stl'] = get_text(player.find('td',{"data-stat": "stl"}))
            game_players['blk'] = get_text(player.find('td',{"data-stat": "blk"}))
            game_players['tov'] = get_text(player.find('td',{"data-stat": "tov"}))
            game_players['pf'] = get_text(player.find('td',{"data-stat": "pf"}))
            game_players['pts'] = get_text(player.find('td',{"data-stat": "pts"}))
            game_players['plus_minus'] = get_text(player.find('td',{"data-stat": "plus_minus"}))
            game_players['player_stat_key'] = game_players['game_key'] + '|' + game_players['player_key'] + '|' + game_players['stat_period'] 
            if cnt <= 5:
                game_players['starter_flag'] = True 
            else:
                game_players['starter_flag'] = False
            game_players['load_datetime'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")  
            #print(game_players)
            player_game_data.append(game_players)
            cnt += 1

    return player_game_data
   
def get_text(stat):
    if stat is not None:
        if stat.text != "":
            txt = stat.text
        else:
            txt = None
    else:
        txt = None
    return txt

In [ ]:
    
    ##########################################################################
    # Input Data Check
    ##########################################################################
    
    try:
        request_json = request
        if request_json and 'StartDate' in request_json:  
            startDate = datetime.strptime(request_json['StartDate'], '%Y-%m-%d').date()            
        else:
            startDate = get_max_game_date()
        if request_json and 'EndDate' in request_json:  
            endDate = datetime.strptime(request_json['EndDate'], '%Y-%m-%d').date() 
        else:
            endDate = (datetime.now() + timedelta(days=-1)).date()
    except Exception as e:
        raise ValueError("Start & End dates must be in YYYY-MM-DD format") from e

    # Distinct list of Months between start and end date
    delta = endDate - startDate       # as timedelta

    if delta.days < 0:
        raise ValueError("Start Date can't be before End Date") 
    
    ##########################################################################
    # Get Distinct Months for schedule to scrape
    ##########################################################################
    
    yearmonths = []
    for i in range(delta.days + 1):
        r = {}
        day = startDate + timedelta(days=i)
        r['monthname'] = day.strftime('%B').lower()
        if day.month > 9:
            r['year'] = day.year + 1
        else:
            r['year'] = day.year
        if r not in yearmonths: 
            yearmonths.append(r)
    #print(yearmonths)

In [ ]:
yearmonths

In [ ]:
    
##########################################################################
# Scrape Schedule
##########################################################################
player_game_rows_loaded = 0
game_rows_loaded = 0


schedule = []
v = yearmonths[12]
year = str(v['year'])
month = v['monthname']
if month == 'october' and (year == '2020' or year == '2021'):
    url = f'https://www.basketball-reference.com/leagues/NBA_{year}_games-{month}-{v["year"] - 1}.html'
else:
    url = f'https://www.basketball-reference.com/leagues/NBA_{year}_games-{month}.html'
print(url)

In [ ]:
html = requests.get(url)

if html.ok:
    soup = BeautifulSoup(html.content, 'html.parser')  
else:
    print(f'No data for {month} {year} because enountered error code {html.status_code}')
    #continue

rows = soup.find('table', id="schedule").find('tbody').find_all('tr')
#print(rows)

In [ ]:
soup

In [ ]:
for row in rows:
    game_date_node = row.find('th',{"data-stat": "date_game"})
    if game_date_node is not None:

        game_date = datetime.strptime(game_date_node.text, '%a, %b %d, %Y').date()
        if game_date >= startDate and game_date <= endDate:
            #cells = row.find_all(['td', 'th'])
            r = {}
            #r.setdefault(game_start_time, []).append(value)

            v1 = row.find('th',{"data-stat": "date_game"})
            #r[k1] = v1.text
            r['game_date'] = datetime.strptime(v1.text, '%a, %b %d, %Y').strftime("%Y-%m-%d")

            v2 = row.find('td',{"data-stat": "game_start_time"})
            r['game_start_time'] = v2.text if v2 else None

            v3 = row.find('td',{"data-stat": "visitor_team_name"})
            r['visitor_team_name'] = v3.text
            r['away_abbr'] = v3['csk'].split('.')[0]

            v4 = row.find('td',{"data-stat": "visitor_pts"})
            r['visitor_pts'] = v4.text if v4 else None

            v5 = row.find('td',{"data-stat": "home_team_name"})
            r['home_team_name'] = v5.text
            r['home_abbr'] = v5['csk'].split('.')[0]

            v6 = row.find('td',{"data-stat": "home_pts"})
            r['home_pts'] = v6.text if v6 else None

            v7 = row.find('td',{"data-stat": "box_score_text"}).find('a',href=True)
            r['box_score_url'] = v7['href'] if v7 else None

            v8 = row.find('td',{"data-stat": "attendance"})
            r['attendance'] = v8.text if v8 else None

            v9 = row.find('td',{"data-stat": "overtimes"})
            r['overtimes'] = v9.text if v9 else None

            if r['game_start_time']:
                v12 = r['away_abbr'] + r['game_date'].replace('-','') + r['home_abbr'] + r['game_start_time'].replace(':','')
            else:
                v12 = r['away_abbr'] + r['game_date'].replace('-','') + r['home_abbr']
            r['game_key'] = v12 if v12 else None

            schedule.append(r)
print(schedule)            

In [ ]:
        
            ##########################################################################
            # Scrape Games in Schedule
            ##########################################################################
            games_data = []
            player_game_data = []

            for game in schedule:
                if 'box_score_url' in game and game['box_score_url'] != "" and game['box_score_url'] is not None:


                    url = "https://www.basketball-reference.com" + game['box_score_url']

                    #print(url)
                    r = requests.get(url)
                    #print('here2')
                    soup = BeautifulSoup(str(r.content).replace("<!--","").replace('-->',''), 'html.parser')

                    ##############################################
                    # Line Score
                    rows = soup.find('table', id="line_score").find_all('tr')

                    # Away Line Score
                    r_num = 1
                    for away in rows[2].find_all('td'):
                        test_strong = away.find('strong') # Strong represents the total score ... ignore
                        if test_strong is None and r_num < 7:
                            k='a_g' + str(r_num) + '_score'
                            game[k] = away.text if away.text != "" else None
                        r_num+=1

                    # Home Line Score
                    r_num = 1
                    for home in rows[3].find_all('td'):
                        test_strong = home.find('strong') # Strong represents the total score ... ignore
                        if test_strong is None and r_num < 7:
                            k='h_g' + str(r_num) + '_score'
                            game[k] = home.text if home.text != "" else None
                        r_num+=1    

                    ##############################################
                    # Four Facts
                    rows = soup.find('table', id="four_factors").find_all('tr')

                    # Away Four Factors
                    game['a_ff_pace'] = rows[2].find('td',{"data-stat": "pace"}).text
                    game['a_ff_efg_pct'] = rows[2].find('td',{"data-stat": "efg_pct"}).text
                    game['a_ff_tov_pct'] = rows[2].find('td',{"data-stat": "tov_pct"}).text
                    game['a_ff_orb_pct'] = rows[2].find('td',{"data-stat": "orb_pct"}).text
                    game['a_ff_ft_rate'] = rows[2].find('td',{"data-stat": "ft_rate"}).text
                    game['a_ff_off_rtg'] = rows[2].find('td',{"data-stat": "off_rtg"}).text

                    # Home Four Factors
                    game['h_ff_pace'] = rows[3].find('td',{"data-stat": "pace"}).text
                    game['h_ff_efg_pct'] = rows[3].find('td',{"data-stat": "efg_pct"}).text
                    game['h_ff_tov_pct'] = rows[3].find('td',{"data-stat": "tov_pct"}).text
                    game['h_ff_orb_pct'] = rows[3].find('td',{"data-stat": "orb_pct"}).text
                    game['h_ff_ft_rate'] = rows[3].find('td',{"data-stat": "ft_rate"}).text
                    game['h_ff_off_rtg'] = rows[3].find('td',{"data-stat": "off_rtg"}).text
                    game['load_datetime'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")        

                    #now = datetime.now() # current date and time
                    #now.strftime("%m/%d/%Y, %H:%M:%S")


                    #player_game_data = []
                    game_date = game['game_date']

                    ##############################################
                    # Game Box - Home
                    #box-WAS-q1-basic
                    stat_type = "game"
                    h_or_a = "h"
                    team_abbrev = game['home_abbr']
                    id_string = "box-" + game['home_abbr'] + "-" + stat_type + "-basic"
                    player_game_data = get_game_players(soup, player_game_data, id_string, game['game_key'], stat_type, h_or_a, team_abbrev, game_date)

                    ##############################################
                    # Game Box - Away
                    #box-WAS-q1-basic
                    stat_type = "game"
                    h_or_a = "a"
                    team_abbrev = game['away_abbr']
                    id_string = "box-" + game['away_abbr'] + "-" + stat_type + "-basic"
                    player_game_data = get_game_players(soup, player_game_data, id_string, game['game_key'], stat_type, h_or_a, team_abbrev, game_date)

#                     ##############################################
#                     # Q1 Box - Home
#                     stat_type = "q1"
#                     h_or_a = "h"
#                     team_abbrev = game['home_abbr']
#                     id_string = "box-" + game['home_abbr'] + "-" + stat_type + "-basic"
#                     player_game_data = get_game_players(soup, player_game_data, id_string, game['game_key'], stat_type, h_or_a, team_abbrev, game_date)

#                     ##############################################
#                     # Q1 Box - Away
#                     stat_type = "q1"
#                     h_or_a = "a"
#                     team_abbrev = game['away_abbr']
#                     id_string = "box-" + game['away_abbr'] + "-" + stat_type + "-basic"
#                     player_game_data = get_game_players(soup, player_game_data, id_string, game['game_key'], stat_type, h_or_a, team_abbrev, game_date)

#                     ##############################################
#                     # Q2 Box - Home
#                     stat_type = "q2"
#                     h_or_a = "h"
#                     team_abbrev = game['home_abbr']
#                     id_string = "box-" + game['home_abbr'] + "-" + stat_type + "-basic"
#                     player_game_data = get_game_players(soup, player_game_data, id_string, game['game_key'], stat_type, h_or_a, team_abbrev, game_date)

#                     ##############################################
#                     # Q2 Box - Away
#                     stat_type = "q2"
#                     h_or_a = "a"
#                     team_abbrev = game['away_abbr']
#                     id_string = "box-" + game['away_abbr'] + "-" + stat_type + "-basic"
#                     player_game_data = get_game_players(soup, player_game_data, id_string, game['game_key'], stat_type, h_or_a, team_abbrev, game_date)

#                     ##############################################
#                     # Q3 Box - Home
#                     stat_type = "q3"
#                     h_or_a = "h"
#                     team_abbrev = game['home_abbr']
#                     id_string = "box-" + game['home_abbr'] + "-" + stat_type + "-basic"
#                     player_game_data = get_game_players(soup, player_game_data, id_string, game['game_key'], stat_type, h_or_a, team_abbrev, game_date)

#                     ##############################################
#                     # Q3 Box - Away
#                     stat_type = "q3"
#                     h_or_a = "a"
#                     team_abbrev = game['away_abbr']
#                     id_string = "box-" + game['away_abbr'] + "-" + stat_type + "-basic"
#                     player_game_data = get_game_players(soup, player_game_data, id_string, game['game_key'], stat_type, h_or_a, team_abbrev, game_date)

#                     ##############################################
#                     # Q4 Box - Home
#                     stat_type = "q4"
#                     h_or_a = "h"
#                     team_abbrev = game['home_abbr']
#                     id_string = "box-" + game['home_abbr'] + "-" + stat_type + "-basic"
#                     player_game_data = get_game_players(soup, player_game_data, id_string, game['game_key'], stat_type, h_or_a, team_abbrev, game_date)

#                     ##############################################
#                     # Q4 Box - Away
#                     stat_type = "q4"
#                     h_or_a = "a"
#                     team_abbrev = game['away_abbr']
#                     id_string = "box-" + game['away_abbr'] + "-" + stat_type + "-basic"
#                     player_game_data = get_game_players(soup, player_game_data, id_string, game['game_key'], stat_type, h_or_a, team_abbrev, game_date)

                    games_data.append(game)

In [ ]:
pandas_games_data = pd.DataFrame(games_data)

In [ ]:
pandas_games_data['game_start_time']

In [ ]:
            ##########################################################################
            # Check for empty game data
            ##########################################################################
            # Continue to next month if there were no games in the month starting at the start date
            if not games_data:
                continue                    
  

In [ ]:
project = 'nba-predictions-dev'

In [ ]:
  
            ##########################################################################
            # Save to BigQuery
            ##########################################################################

            # print(player_game_data)
            # print(games_data)

            # Config
            client = bigquery.Client(project=project)
            print(f'Loading data for {month} {year}')
            
            #player game data
            pandas_player_game_data = pd.DataFrame(player_game_data)
            pandas_player_game_data['game_date'] = pandas_player_game_data['game_date'].astype('datetime64[ns]')
            pandas_player_game_data['load_datetime'] = pandas_player_game_data['load_datetime'].astype('datetime64[ns]')
            job_config = bigquery.LoadJobConfig()
            job_config.autodetect='True'
            job_config.create_disposition = 'CREATE_IF_NEEDED'
            job_config.write_disposition = 'WRITE_APPEND'
             ## Set schema for specific columns where more information is needed (e.g. not NULLABLE or specific date/time)
            job_config.schema = [
                bigquery.SchemaField('player_stat_key','STRING', 'REQUIRED'),
                bigquery.SchemaField('game_date','DATE'),
                bigquery.SchemaField('load_datetime','TIMESTAMP'),
                bigquery.SchemaField('starter_flag','BOOL')
            ]
            job_config.time_partitioning = bigquery.TimePartitioning(
                type_=bigquery.TimePartitioningType.DAY,
                field="game_date")
            job_player = client.load_table_from_dataframe(pandas_player_game_data, 'nba.raw_basketballreference_playerbox' \
                                                          , job_config=job_config, project=project)
            player_result = job_player.result()
            player_message = (
                f'Job ID: {player_result.job_id} '
                f'was started {player_result.started} '
                f'and ended {player_result.ended} '
                f'loading {player_result.output_rows} row(s) '
                f'to {player_result.destination}')
            print(player_message)
            player_game_rows_loaded = player_game_rows_loaded + player_result.output_rows

            #game data
            pandas_games_data = pd.DataFrame(games_data)
            pandas_games_data['game_date'] = pandas_games_data['game_date'].astype('datetime64[ns]')
            pandas_games_data['load_datetime'] = pandas_games_data['load_datetime'].astype('datetime64[ns]')
            job_config = bigquery.LoadJobConfig()
            job_config.autodetect='True'
            job_config.create_disposition = 'CREATE_IF_NEEDED'
            job_config.write_disposition = 'WRITE_APPEND'
            ## Set schema for specific columns where more information is needed (e.g. not NULLABLE or specific date/time)
            job_config.schema = [
                bigquery.SchemaField('game_key','STRING', 'REQUIRED'),
                bigquery.SchemaField('game_date','STRING', 'REQUIRED'),
                bigquery.SchemaField('home_team_name','STRING', 'REQUIRED'),
                bigquery.SchemaField('home_abbr','STRING', 'REQUIRED'),
                bigquery.SchemaField('visitor_team_name','STRING', 'REQUIRED'),
                bigquery.SchemaField('away_abbr','STRING', 'REQUIRED'),
                bigquery.SchemaField('game_date','DATE'),
                bigquery.SchemaField('load_datetime','TIMESTAMP'),
            ]
            job_config.time_partitioning = bigquery.TimePartitioning(
                type_=bigquery.TimePartitioningType.DAY,
                field="game_date")
            job_game = client.load_table_from_dataframe(pandas_games_data, 'nba.raw_basketballreference_game', \
                                                        job_config=job_config, project=project)
            game_result = job_game.result()
            game_message = (
                f'Job ID: {game_result.job_id} '
                f'was started {game_result.started} '
                f'and ended {game_result.ended} '
                f'loading {game_result.output_rows} row(s) '
                f'to {game_result.destination}')
            print(game_message)
            game_rows_loaded = game_rows_loaded + game_result.output_rows

            print(f'Successfully loaded {player_game_rows_loaded} row(s) to raw_basketballreference_playerbox and {game_rows_loaded} to raw_basketballreference_game')

In [ ]:
import requests
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import pandas as pd
from google.cloud import storage


def get_games(startDate,endDate):
    ##########################################################################
    # Get Distinct Months for schedule to scrape
    ##########################################################################

    delta = endDate - startDate
    
    yearmonths = []
    for i in range(delta.days + 1):
        r = {}
        day = startDate + timedelta(days=i)
        r['monthname'] = day.strftime('%B').lower()
        if day.month > 9:
            r['year'] = day.year + 1
        else:
            r['year'] = day.year
        if r not in yearmonths: 
            yearmonths.append(r)

    schedule = []
    for v in yearmonths:
        year = str(v['year'])
        month = v['monthname']
        url = 'https://www.basketball-reference.com/leagues/NBA_' + year + '_games-' + month + '.html'
        #print(url)

        html = requests.get(url)

        if html.ok:
            soup = BeautifulSoup(html.content, 'html.parser')  
        else:
            print(f'No data for {month} {year} because enountered error code {html.status_code}')
            continue

        rows = soup.find('table', id="schedule").find('tbody').find_all('tr')

        for row in rows:
            game_date_node = row.find('th',{"data-stat": "date_game"})
            if game_date_node is not None:

                game_date = datetime.strptime(game_date_node.text, '%a, %b %d, %Y').date()
                if game_date >= startDate and game_date <= endDate:
                    #cells = row.find_all(['td', 'th'])
                    r = {}
                    #r.setdefault(game_start_time, []).append(value)

                    v1 = row.find('th',{"data-stat": "date_game"})
                    #r[k1] = v1.text
                    r['game_date'] = datetime.strptime(v1.text, '%a, %b %d, %Y').strftime("%Y-%m-%d")
                    r['game_day'] = datetime.strptime(v1.text, '%a, %b %d, %Y').strftime("%A")

                    v2 = row.find('td',{"data-stat": "game_start_time"})
                    r['game_start_time'] = v2.text if v2 else None

                    v3 = row.find('td',{"data-stat": "visitor_team_name"})
                    r['visitor_team_name'] = v3.text
                    r['away_abbr'] = v3['csk'].split('.')[0]

                    v4 = row.find('td',{"data-stat": "home_team_name"})
                    r['home_team_name'] = v4.text
                    r['home_abbr'] = v4['csk'].split('.')[0]

                    if r['game_start_time']:
                        v12 = r['away_abbr'] + r['game_date'].replace('-','') + r['home_abbr'] + r['game_start_time'].replace(':','')
                    else:
                        v12 = r['away_abbr'] + r['game_date'].replace('-','') + r['home_abbr']
                    r['game_key'] = v12 if v12 else None

                    schedule.append(r)
                
    return schedule

def write_to_bucket(request):
    
    try:
        if type(request) == 'dict':
            request_json = request
        else:
            request_json = request.get_json()      
        if request_json and 'ScheduleDays' in request_json:
            schedule_days = request_json['ScheduleDays']
        else:
            schedule_days = 14
    except Exception as e:
        raise ValueError("Invalid input. Please provide ScheduleDays as an integer") from e
    
    startDate = (datetime.now()).date()
    endDate = (startDate + timedelta(days=schedule_days))
    schedule = get_games(startDate,endDate) 
    
    game_date = pd.DataFrame(schedule)
    client = storage.Client()
    bucket_name = os.environ.get("CLOUD_STORAGE_BUCKET")
    bucket = client.bucket(bucket_name)
    bucket.blob('static/upcoming.json').upload_from_string(game_date.to_json(), 'text/json')


In [ ]:
schedule

In [ ]:
game_date

In [ ]:
from google.cloud import storage
from io import StringIO # if going with no saving csv file

In [ ]:
from google.cloud import storage



 

In [ ]:
bucket.blob('static/monday.csv').upload_from_string(game_date[game_date['game_day']=='Monday'].to_csv(), 'text/csv')
bucket.blob('static/tuesday.csv').upload_from_string(game_date[game_date['game_day']=='Tuesday'].to_csv(), 'text/csv')
bucket.blob('static/wednesday.csv').upload_from_string(game_date[game_date['game_day']=='Wednesday'].to_csv(), 'text/csv')
bucket.blob('static/thursday.csv').upload_from_string(game_date[game_date['game_day']=='Thursday'].to_csv(), 'text/csv')
bucket.blob('static/friday.csv').upload_from_string(game_date[game_date['game_day']=='Friday'].to_csv(), 'text/csv')
bucket.blob('static/saturday.csv').upload_from_string(game_date[game_date['game_day']=='Saturday'].to_csv(), 'text/csv')
bucket.blob('static/sunday.csv').upload_from_string(game_date[game_date['game_day']=='Sunday'].to_csv(), 'text/csv')

In [ ]:
game_date[game_date['game_day']=='Monday']

In [ ]:
bucket.blob('static/monday.json').upload_from_string(game_date[game_date['game_day']=='Monday'].to_json(), 'text/json')
bucket.blob('static/tuesday.json').upload_from_string(game_date[game_date['game_day']=='Tuesday'].to_json(), 'text/json')
bucket.blob('static/wednesday.json').upload_from_string(game_date[game_date['game_day']=='Wednesday'].to_json(), 'text/json')
bucket.blob('static/thursday.json').upload_from_string(game_date[game_date['game_day']=='Thursday'].to_json(), 'text/json')
bucket.blob('static/friday.json').upload_from_string(game_date[game_date['game_day']=='Friday'].to_json(), 'text/json')
bucket.blob('static/saturday.json').upload_from_string(game_date[game_date['game_day']=='Saturday'].to_json(), 'text/json')
bucket.blob('static/sunday.json').upload_from_string(game_date[game_date['game_day']=='Sunday'].to_json(), 'text/json')

In [ ]:
bucket = client.bucket('nba-predictions-dev.appspot.com')

In [ ]:
d_cpt = StringIO(bucket.blob('static/monday.json').download_as_string())
df = pd.read_csv(d_cpt)
df

In [ ]:
    data = json.loads(blob.decode("utf-8").replace("'",'"'))
    home_teams = list(data['home_team_name'].values())
    away_teams = list(data['visitor_team_name'].values())
    game_day = list(data['game_day'].values())
    game_date = list(data['game_date'].values())
    game_start_time = list(data['game_start_time'].values())

In [ ]:
games = []
for i in range(len(home_teams)-1):
    games.append(f'{away_teams[i]} vs. {home_teams[i]} at {game_start_time[i]} on {game_day[i]}, {game_date[i]}')
games

In [ ]:
home_teams[6]

In [ ]:
range(len(home_teams)-1)

In [ ]:
bucket

In [ ]:
blob = bucket.blob('static/monday.json').download_as_string()

In [ ]:
blob

In [ ]:
import json

In [ ]:
dict = json.loads(blob.decode("utf-8").replace("'",'"'))

In [ ]:
dict

In [ ]:
list(dict['home_team_name'].values())

In [ ]:
list(dict['visitor_team_name'].values())

In [ ]:
blob.get_json()

In [ ]:
blob.download_as_string()

In [ ]:
blob2 = bucket.blob('monday.json')

In [ ]:
blob2

In [169]:
W = 10

In [170]:
type(W)

int